In [ ]:
%%bash 

cd src/

make

In [ ]:
%%bash

cat model/test_product.ma

In [ ]:
%%bash

cat model/test_product.ev

In [ ]:
%%bash

src/bin/cd++ -m model/test_product.ma \
             -t 10:00:00:000 \
             -l model.log \
             -o model.out \
             -e model/test_product.ev

In [ ]:
%%bash

cat model.out